Notebook for checking
1) Stability to biomass perturbations and
2) Stability to invasion
for the steady state GNE produced in the 
notebook "ecoli_model_compute_ss_gne.ipynb".

In [1]:
import numpy as np
import scipy.linalg as linalg
import scipy.sparse as sparse
import scipy.sparse.linalg as sp_linalg
import matplotlib.pyplot as plt
import scipy.io as sio
import cvxpy as cp
import time
import gurobipy
import pickle


In [2]:
# Set solver to use with cvxpy. Currently, computing whether or not a steady state GNE 
# is stable (to biomass perturbations or to invasion) requires Gurobi, as 
# the Gurobi interface is used extract information related to dual variables of 
# the linear programs. See here for how to set up a free academic Gurobi 
# license: https://www.gurobi.com/features/academic-named-user-license/.
cp_solver = 'GUROBI'


In [3]:
# Load data for model.
directory = '../ModelFiles/FourSpecies'

# S contains the stoichiometry matrices R_{k} and R_{k}^{ex} for each species k.
S = sio.loadmat(directory + '/S.mat')['S']
I = sio.loadmat(directory + '/I.mat')['I'][0][0]
J = sio.loadmat(directory + '/J.mat')['J'][0][0]
reaction_lb = sio.loadmat(directory + '/lb.mat')['lb']
reaction_ub = sio.loadmat(directory + '/ub.mat')['ub']

# Indices of reactions and metabolites for each species, needed because 
# cobra groups all metabolites and reactions into a single model.
lumen_reactions_idx = sio.loadmat(directory + '/lumen_reactions_idx.mat')['lumen_reactions_idx'] - 1
lumen_metabolites_idx = sio.loadmat(directory + '/lumen_metabolites_idx.mat')['lumen_metabolites_idx'] - 1
lumen_reaction_names = sio.loadmat(directory + '/lumen_reactions.mat')['lumen_reactions']

Ec1_reactions_idx = sio.loadmat(directory + '/Ec1_reactions_idx.mat')['Ec1_reactions_idx'] - 1
Ec1_reaction_names = sio.loadmat(directory + '/Ec1_reactions.mat')['Ec1_reactions']
Ec1_metabolites_idx = sio.loadmat(directory + '/Ec1_metabolites_idx.mat')['Ec1_metabolites_idx'] - 1
Ec1_biomass_idx = sio.loadmat(directory + '/Ec1_biomass_idx.mat')['Ec1_biomass_idx'][0][0]-1

Ec2_reactions_idx = sio.loadmat(directory + '/Ec2_reactions_idx.mat')['Ec2_reactions_idx'] - 1
Ec2_reaction_names = sio.loadmat(directory + '/Ec2_reactions.mat')['Ec2_reactions']
Ec2_metabolites_idx = sio.loadmat(directory + '/Ec2_metabolites_idx.mat')['Ec2_metabolites_idx'] - 1
Ec2_biomass_idx = sio.loadmat(directory + '/Ec2_biomass_idx.mat')['Ec2_biomass_idx'][0][0]-1

Ec3_reactions_idx = sio.loadmat(directory + '/Ec3_reactions_idx.mat')['Ec3_reactions_idx'] - 1
Ec3_reaction_names = sio.loadmat(directory + '/Ec3_reactions.mat')['Ec3_reactions']
Ec3_metabolites_idx = sio.loadmat(directory + '/Ec3_metabolites_idx.mat')['Ec3_metabolites_idx'] - 1
Ec3_biomass_idx = sio.loadmat(directory + '/Ec3_biomass_idx.mat')['Ec3_biomass_idx'][0][0]-1

Ec4_reactions_idx = sio.loadmat(directory + '/Ec4_reactions_idx.mat')['Ec4_reactions_idx'] - 1
Ec4_reaction_names = sio.loadmat(directory + '/Ec4_reactions.mat')['Ec4_reactions']
Ec4_metabolites_idx = sio.loadmat(directory + '/Ec4_metabolites_idx.mat')['Ec4_metabolites_idx'] - 1
Ec4_biomass_idx = sio.loadmat(directory + '/Ec4_biomass_idx.mat')['Ec4_biomass_idx'][0][0]-1

I1 = len(Ec1_metabolites_idx); I2 = len(Ec2_metabolites_idx); I3 = len(Ec3_metabolites_idx); I4 = len(Ec4_metabolites_idx)
Jl = len(lumen_reactions_idx); J1 = len(Ec1_reactions_idx); J2 = len(Ec2_reactions_idx); J3 = len(Ec3_reactions_idx); J4 = len(Ec4_reactions_idx)

Ec1_reaction_names = np.array([Ec1_reaction_names[i][0] for i in range(len(Ec1_reaction_names))])
Ec2_reaction_names = np.array([Ec2_reaction_names[i][0] for i in range(len(Ec2_reaction_names))])
Ec3_reaction_names = np.array([Ec3_reaction_names[i][0] for i in range(len(Ec3_reaction_names))])
Ec4_reaction_names = np.array([Ec4_reaction_names[i][0] for i in range(len(Ec4_reaction_names))])
lumen_reaction_names = np.array([lumen_reaction_names[i][0] for i in range(len(lumen_reaction_names))])

# Create vectors that can be dotted with vector of reactions for each species 
# and pull out the biomass reaction.
e1 = sparse.identity(J1 + Jl).tocsr()[:, Ec1_biomass_idx]; e2 = sparse.identity(J2 + Jl).tocsr()[:, Ec2_biomass_idx]
e3 = sparse.identity(J3 + Jl).tocsr()[:, Ec3_biomass_idx]; e4 = sparse.identity(J4 + Jl).tocsr()[:, Ec4_biomass_idx]


In [4]:
# Load steady state GNE generated in the notebook titled 'ecoli_model_compute_ss_gne.ipynb'.
# steady_states.p corresponds corresponds to results for dilution rate equal to 0.5, 
# steady_states_040.p to results for dilution rate equal to 0.4, 
# steady_states_060.p to results for diluation rate equal to 0.6,
# and steady_states_0018.p to results for dilution rate equal to 0.018.
death_rate = np.array([[0.5]])
steady_states = pickle.load(open("steady_states.p", "rb"))


In [6]:
def stability(x1, bm1, x2, bm2, x3, bm3, x4, bm4, pert_size):
    ''' Function for computing whether or not the steady state GNE with biomasses [bm1, bm2, bm3, bm4] and fluxes 
    [x1, x2, x3, x4] is stable. If unstable, the function also provides the size ofthe maximum eigenvalue and its 
    corresponding eigenvector, which gives an unstable perturbation direction. pert_size is the size of the 
    perturbations of the upper and lower reaction bounds used to ensure uniqueness of the dual solutions. '''

    print('Starting stability function')
    # Re-run best response problem with Gurobi/simplex method to get optimal basis.
    
    env = gurobipy.Env(empty=True)
    env.setParam("OutputFlag",0)
    env.start()
    
    # Ec1's problem.
    m1 = gurobipy.Model("Ec1", env=env)

    # Create variables.
    lb1 = reaction_lb[Ec1_reactions_idx.flatten()].flatten()
    ub1 = reaction_ub[Ec1_reactions_idx.flatten()].flatten()
    
    lb1_pert = np.random.random((J1, 1))
    lbl_pert = np.random.random((Jl, 1))
    ub1_pert = np.random.random((J1, 1))
    
    lb1 = lb1 - pert_size * lb1_pert.flatten()
    ub1 = ub1 + pert_size * ub1_pert.flatten()

    rxns1 = m1.addMVar(shape=J1, name='rxns1', lb=lb1, ub=ub1);

    lb1_ex = (reaction_lb[lumen_reactions_idx.flatten()].flatten() - bm2 * x2[J2:].flatten() - bm3 * x3[J3:].flatten() - bm4 * x4[J4:].flatten()) / bm1
    ub1_ex = (reaction_ub[lumen_reactions_idx.flatten()].flatten() - bm2 * x2[J2:].flatten() - bm3 * x3[J3:].flatten() - bm4 * x4[J4:].flatten()) / bm1
    
    lb1_ex = lb1_ex - pert_size * lbl_pert.flatten()
    
    rxns1_ex = m1.addMVar(shape=Jl, name='rxns1_ex', lb=lb1_ex, ub=ub1_ex);

    # Set objective.
    m1.setObjective(e1.toarray()[0:J1].T @ rxns1, gurobipy.GRB.MAXIMIZE);

    # Make constraints.
    S1 = S[np.concatenate([Ec1_metabolites_idx, lumen_metabolites_idx]).flatten(), :]
    S1_ex = S1[:, lumen_reactions_idx.flatten()]
    S1 = S1[:, Ec1_reactions_idx.flatten()]
    m1.addConstr(S1 @ rxns1 + S1_ex @ rxns1_ex == np.zeros((S1.shape[0],)), name='internal_fba1');

    # Solve.
    m1.params.Method = 0;
    m1.update();
    m1.optimize();
    
    # Ec2's problem.
    m2 = gurobipy.Model("Ec2", env=env)

    # Create variables.
    lb2 = reaction_lb[Ec2_reactions_idx.flatten()].flatten()
    ub2 = reaction_ub[Ec2_reactions_idx.flatten()].flatten()
    
    lb2_pert = np.random.random((J2, 1))
    lbl_pert = np.random.random((Jl, 1))
    ub2_pert = np.random.random((J2, 1))
    
    lb2 = lb2 - pert_size * lb2_pert.flatten()
    ub2 = ub2 + pert_size * ub2_pert.flatten()
    
    rxns2 = m2.addMVar(shape=J2, name='rxns2', lb=lb2, ub=ub2);

    lb2_ex = (reaction_lb[lumen_reactions_idx.flatten()].flatten() - bm1 * x1[J1:].flatten() - bm3 * x3[J3:].flatten() - bm4 * x4[J4:].flatten()) / bm2
    ub2_ex = (reaction_ub[lumen_reactions_idx.flatten()].flatten() - bm1 * x1[J1:].flatten() - bm3 * x3[J3:].flatten() - bm4 * x4[J4:].flatten()) / bm2
    
    lb2_ex = lb2_ex - pert_size * lbl_pert.flatten()
    
    rxns2_ex = m2.addMVar(shape=Jl, name='rxns2_ex', lb=lb2_ex, ub=ub2_ex);

    # Set objective.
    m2.setObjective(e2.toarray()[0:J2].T @ rxns2, gurobipy.GRB.MAXIMIZE);

    # Make constraints.
    S2 = S[np.concatenate([Ec2_metabolites_idx, lumen_metabolites_idx]).flatten(), :]
    S2_ex = S2[:, lumen_reactions_idx.flatten()]
    S2 = S2[:, Ec2_reactions_idx.flatten()]
    m2.addConstr(S2 @ rxns2 + S2_ex @ rxns2_ex == np.zeros((S2.shape[0],)), name='internal_fba2');

    m2.params.Method = 0;
    m2.update();
    m2.optimize();
    
    # Ec3's problem.
    m3 = gurobipy.Model("Ec3", env=env)

    # Create variables.
    lb3 = reaction_lb[Ec3_reactions_idx.flatten()].flatten()
    ub3 = reaction_ub[Ec3_reactions_idx.flatten()].flatten()
    
    lb3_pert = np.random.random((J3,1))
    lbl_pert = np.random.random((Jl,1))
    ub3_pert = np.random.random((J3,1))
    
    lb3 = lb3 - pert_size * lb3_pert.flatten()
    ub3 = ub3 + pert_size * ub3_pert.flatten()
    
    rxns3 = m3.addMVar(shape=J3, name='rxns3', lb=lb3, ub=ub3);

    lb3_ex = (reaction_lb[lumen_reactions_idx.flatten()].flatten() - bm2 * x2[J2:].flatten() - bm1 * x1[J1:].flatten() - bm4 * x4[J4:].flatten()) / bm3
    ub3_ex = (reaction_ub[lumen_reactions_idx.flatten()].flatten() - bm2 * x2[J2:].flatten() - bm1 * x1[J1:].flatten() - bm4 * x4[J4:].flatten()) / bm3
    
    lb3_ex = lb3_ex - pert_size * lbl_pert.flatten()
    
    rxns3_ex = m3.addMVar(shape=Jl, name='rxns3_ex', lb=lb3_ex, ub=ub3_ex);

    # Set objective.
    m3.setObjective(e3.toarray()[0:J3].T @ rxns3, gurobipy.GRB.MAXIMIZE);

    # Make constraints.
    S3 = S[np.concatenate([Ec3_metabolites_idx, lumen_metabolites_idx]).flatten(), :]
    S3_ex = S3[:, lumen_reactions_idx.flatten()]
    S3 = S3[:, Ec3_reactions_idx.flatten()]
    m3.addConstr(S3 @ rxns3 + S3_ex @ rxns3_ex == np.zeros((S3.shape[0],)), name='internal_fba3');

    m3.params.Method = 0;
    m3.update();
    m3.optimize();
    
    # Ec4's problem.
    m4 = gurobipy.Model("Ec4", env=env)

    # Create variables.
    lb4 = reaction_lb[Ec4_reactions_idx.flatten()].flatten()
    ub4 = reaction_ub[Ec4_reactions_idx.flatten()].flatten()
    
    lb4_pert = np.random.random((J4,1))
    lbl_pert = np.random.random((Jl,1))
    ub4_pert = np.random.random((J4,1))
    
    lb4 = lb4 - pert_size * lb4_pert.flatten()
    ub4 = ub4 + pert_size * ub4_pert.flatten()
    
    rxns4 = m4.addMVar(shape=J4, name='rxns4', lb=lb4, ub=ub4);

    lb4_ex = (reaction_lb[lumen_reactions_idx.flatten()].flatten() - bm2 * x2[J2:].flatten() - bm3 * x3[J3:].flatten() - bm1 * x1[J1:].flatten()) / bm4
    ub4_ex = (reaction_ub[lumen_reactions_idx.flatten()].flatten() - bm2 * x2[J2:].flatten() - bm3 * x3[J3:].flatten() - bm1 * x1[J1:].flatten()) / bm4
    
    lb4_ex = lb4_ex - pert_size * lbl_pert.flatten()
    
    rxns4_ex = m4.addMVar(shape=Jl, name='rxns4_ex', lb=lb4_ex, ub=ub4_ex);

    # Set objective.
    m4.setObjective(e4.toarray()[0:J4].T @ rxns4, gurobipy.GRB.MAXIMIZE);

    # Make constraints.
    S4 = S[np.concatenate([Ec4_metabolites_idx, lumen_metabolites_idx]).flatten(), :]
    S4_ex = S4[:, lumen_reactions_idx.flatten()]
    S4 = S4[:, Ec4_reactions_idx.flatten()]
    m4.addConstr(S4 @ rxns4 + S4_ex @ rxns4_ex == np.zeros((S4.shape[0],)), name='internal_fba4');

    m4.params.Method = 0;
    m4.update();
    m4.optimize();
    
    # Check degeneracy.
    primal_basis1 = rxns1.VBasis == 0
    primal_basis1_ex = rxns1_ex.VBasis == 0
    
    active_basic1_lb = np.where(rxns1.X[primal_basis1] - lb1[primal_basis1] == 0)[0]
    active_basic1_ub = np.where(ub1[primal_basis1] - rxns1.X[primal_basis1] == 0)[0]
    active_basic1_ex_lb = np.where(rxns1_ex.X[primal_basis1_ex] - lb1_ex[primal_basis1_ex] == 0)[0]
    active_basic1_ex_ub = np.where(ub1_ex[primal_basis1_ex] - rxns1_ex.X[primal_basis1_ex] == 0)[0]
    
    if len(active_basic1_lb) == 0 and len(active_basic1_ub) == 0 and len(active_basic1_ex_lb) == 0 and len(active_basic1_ex_ub) == 0:
        degenerate = False
    else:
        degenerate = True
        
    if degenerate:
        raise Exception('Ec1 problem is degenerate')
    
    primal_basis2 = rxns2.VBasis == 0
    primal_basis2_ex = rxns2_ex.VBasis == 0
    
    active_basic2_lb = np.where(rxns2.X[primal_basis2] - lb2[primal_basis2] == 0)[0]
    active_basic2_ub = np.where(ub2[primal_basis2] - rxns2.X[primal_basis2] == 0)[0]
    active_basic2_ex_lb = np.where(rxns2_ex.X[primal_basis2_ex] - lb2_ex[primal_basis2_ex] == 0)[0]
    active_basic2_ex_ub = np.where(ub2_ex[primal_basis2_ex] - rxns2_ex.X[primal_basis2_ex] == 0)[0]
    
    if len(active_basic2_lb) == 0 and len(active_basic2_ub) == 0 and len(active_basic2_ex_lb) == 0 and len(active_basic2_ex_ub) == 0:
        degenerate = False
    else:
        degenerate = True
        
    if degenerate:
        raise Exception('Ec2 problem is degenerate')
    
    primal_basis3 = rxns3.VBasis == 0
    primal_basis3_ex = rxns3_ex.VBasis == 0
    
    active_basic3_lb = np.where(rxns3.X[primal_basis3] - lb3[primal_basis3] == 0)[0]
    active_basic3_ub = np.where(ub3[primal_basis3] - rxns3.X[primal_basis3] == 0)[0]
    active_basic3_ex_lb = np.where(rxns3_ex.X[primal_basis3_ex] - lb3_ex[primal_basis3_ex] == 0)[0]
    active_basic3_ex_ub = np.where(ub3_ex[primal_basis3_ex] - rxns3_ex.X[primal_basis3_ex] == 0)[0]
    
    if len(active_basic3_lb) == 0 and len(active_basic3_ub) == 0 and len(active_basic3_ex_lb) == 0 and len(active_basic3_ex_ub) == 0:
        degenerate = False
    else:
        degenerate = True
        
    if degenerate:
        raise Exception('Ec3 problem is degenerate')
    
    primal_basis4 = rxns4.VBasis == 0
    primal_basis4_ex = rxns4_ex.VBasis == 0
    
    active_basic4_lb = np.where(rxns4.X[primal_basis4] - lb4[primal_basis4] == 0)[0]
    active_basic4_ub = np.where(ub4[primal_basis4] - rxns4.X[primal_basis4] == 0)[0]
    active_basic4_ex_lb = np.where(rxns4_ex.X[primal_basis4_ex] - lb4_ex[primal_basis4_ex] == 0)[0]
    active_basic4_ex_ub = np.where(ub4_ex[primal_basis4_ex] - rxns4_ex.X[primal_basis4_ex] == 0)[0]
    
    if len(active_basic4_lb) == 0 and len(active_basic4_ub) == 0 and len(active_basic4_ex_lb) == 0 and len(active_basic4_ex_ub) == 0:
        degenerate = False
    else:
        degenerate = True
        
    if degenerate:
        raise Exception('Ec4 problem is degenerate')

    E = np.where(bm1 * rxns1_ex.X + bm2 * rxns2_ex.X + bm3 * rxns3_ex.X + bm4 * rxns4_ex.X - reaction_lb[lumen_reactions_idx.flatten()].flatten() < 1e-6)[0]
    
    U1 = np.where(ub1 - rxns1.X < 1e-6)[0]
    L1 = np.where(rxns1.X - lb1 < 1e-6)[0]
    F1 = np.where(np.logical_and(~np.isin(np.arange(J1), U1), ~np.isin(np.arange(J1), L1)))[0]
    
    U2 = np.where(ub2 - rxns2.X < 1e-6)[0]
    L2 = np.where(rxns2.X - lb2 < 1e-6)[0]
    F2 = np.where(np.logical_and(~np.isin(np.arange(J2), U2), ~np.isin(np.arange(J2), L2)))[0]
    
    U3 = np.where(ub3 - rxns3.X < 1e-6)[0]
    L3 = np.where(rxns3.X - lb3 < 1e-6)[0]
    F3 = np.where(np.logical_and(~np.isin(np.arange(J3), U3), ~np.isin(np.arange(J3), L3)))[0]
    
    U4 = np.where(ub4 - rxns4.X < 1e-6)[0]
    L4 = np.where(rxns4.X - lb4 < 1e-6)[0]
    F4 = np.where(np.logical_and(~np.isin(np.arange(J4), U4), ~np.isin(np.arange(J4), L4)))[0]

    lambda1 = rxns1_ex.RC
    lambda2 = rxns2_ex.RC
    lambda3 = rxns3_ex.RC
    lambda4 = rxns4_ex.RC
    lambdas = [lambda1, lambda2, lambda3, lambda4]
    biomasses = [bm1, bm2, bm3, bm4]
    
    lambda1_L = np.zeros((len(rxns1.X,)))
    lambda1_L[np.where(rxns1.VBasis == -1)[0]] = rxns1.RC[np.where(rxns1.VBasis == -1)[0]]
    lambda1_U = np.zeros((len(rxns1.X,)))
    lambda1_U[np.where(rxns1.VBasis == -2)[0]] = rxns1.RC[np.where(rxns1.VBasis == -2)[0]]
    
    lambda2_L = np.zeros((len(rxns2.X,)))
    lambda2_L[np.where(rxns2.VBasis == -1)[0]] = rxns2.RC[np.where(rxns2.VBasis == -1)[0]]
    lambda2_U = np.zeros((len(rxns2.X,)))
    lambda2_U[np.where(rxns2.VBasis == -2)[0]] = rxns2.RC[np.where(rxns2.VBasis == -2)[0]]
    
    lambda3_L = np.zeros((len(rxns3.X,)))
    lambda3_L[np.where(rxns3.VBasis == -1)[0]] = rxns3.RC[np.where(rxns3.VBasis == -1)[0]]
    lambda3_U = np.zeros((len(rxns3.X,)))
    lambda3_U[np.where(rxns3.VBasis == -2)[0]] = rxns3.RC[np.where(rxns3.VBasis == -2)[0]]
    
    lambda4_L = np.zeros((len(rxns4.X,)))
    lambda4_L[np.where(rxns4.VBasis == -1)[0]] = rxns4.RC[np.where(rxns4.VBasis == -1)[0]]
    lambda4_U = np.zeros((len(rxns4.X,)))
    lambda4_U[np.where(rxns4.VBasis == -2)[0]] = rxns4.RC[np.where(rxns4.VBasis == -2)[0]]
    
    lambdas_L = [lambda1_L, lambda2_L, lambda3_L, lambda4_L]
    lambdas_U = [lambda1_U, lambda2_U, lambda3_U, lambda4_U]
    
    x1 = np.zeros((x1.shape[0],))
    x1[0:J1] = rxns1.X
    x1[J1:] = rxns1_ex.X
    
    x2 = np.zeros((x2.shape[0],))
    x2[0:J2] = rxns2.X
    x2[J2:] = rxns2_ex.X
    
    x3 = np.zeros((x3.shape[0],))
    x3[0:J3] = rxns3.X
    x3[J3:] = rxns3_ex.X
    
    x4 = np.zeros((x4.shape[0],))
    x4[0:J4] = rxns4.X
    x4[J4:] = rxns4_ex.X
    
    x_values = [x1, x2, x3, x4]
    
    I_values = [I1, I2, I3, I4]
    J_values = [J1, J2, J3, J4]
    
    b1 = (1/bm1) * (reaction_lb[lumen_reactions_idx.flatten()].flatten() - (bm2 * x_values[1].flatten()[J_values[1]:] + bm3 * x_values[2].flatten()[J_values[2]:] + bm4 * x_values[3].flatten()[J_values[3]:]))
    b2 = (1/bm2) * (reaction_lb[lumen_reactions_idx.flatten()].flatten() - (bm1 * x_values[0].flatten()[J_values[0]:] + bm3 * x_values[2].flatten()[J_values[2]:] + bm4 * x_values[3].flatten()[J_values[3]:]))
    b3 = (1/bm3) * (reaction_lb[lumen_reactions_idx.flatten()].flatten() - (bm2 * x_values[1].flatten()[J_values[1]:] + bm1 * x_values[0].flatten()[J_values[0]:] + bm4 * x_values[3].flatten()[J_values[3]:]))
    b4 = (1/bm4) * (reaction_lb[lumen_reactions_idx.flatten()].flatten() - (bm2 * x_values[1].flatten()[J_values[1]:] + bm3 * x_values[2].flatten()[J_values[2]:] + bm1 * x_values[0].flatten()[J_values[0]:]))
    b_values = [b1, b2, b3, b4]
                    
    metabolite_indices = [Ec1_metabolites_idx.flatten(), Ec2_metabolites_idx.flatten(), Ec3_metabolites_idx.flatten(), Ec4_metabolites_idx.flatten()]
    reaction_indices = [Ec1_reactions_idx.flatten(), Ec2_reactions_idx.flatten(), Ec3_reactions_idx.flatten(), Ec4_reactions_idx.flatten()]

    free_var_indices = [F1, F2, F3, F4]
    
    num_vars = 4*4 + 4*4*Jl + 4*np.sum([free_var_indices[l].shape[0] for l in range(4)])
    num_constraints = 4*4 + 4*4*E.shape[0] + 4*np.sum(I_values) + 4*4*Jl
    
    A_stable = sparse.csr_matrix((num_constraints, num_vars))
    b_stable = np.zeros((num_constraints,))
    
    # Make constraints for partials of h_{k}(x) values.
    for k in range(4):
        for j in range(4):
            row_kj = np.zeros((num_vars,))
            row_kj[4*k + j] = 1
            for l in range(4):
                if l != k:
                    row_kj[(4*4 + (4*l + j)*Jl):(4*4 + (4*l + j + 1)*Jl)] = (biomasses[l] / biomasses[k]) * lambdas[k].T

            A_stable[4*k + j, :] = row_kj

            if j == k:
                b_stable[4*k + j] = -(1/biomasses[k]) * lambdas[k].T.dot(b_values[k].flatten())
            else:
                b_stable[4*k + j] = -(1/biomasses[k]) * lambdas[k].T.dot(x_values[j][J_values[j]:])
    # Make constraints for partials of R_{k}^{E} \nu_{k}^{ex} with respect to x_{j}.
    for k in range(4):
        for j in range(4):
            block_kj = np.zeros((E.shape[0], num_vars))
            block_kj[:, 4*4 + (4*k + j)*Jl + E] = np.eye(E.shape[0])

            for l in range(4):
                if l != k:
                    block_kj[:, 4*4 + (4*l + j)*Jl + E] = (biomasses[l] / biomasses[k]) * np.eye(E.shape[0])

            A_stable[(4*4 + (4*k + j)*E.shape[0]):(4*4 + (4*k + j + 1)*E.shape[0]), :] = sparse.csr_matrix(block_kj)

            if j == k:
                b_stable[(4*4 + (4*k + j)*E.shape[0]):(4*4 + (4*k + j + 1)*E.shape[0])] = -(1 / biomasses[k]) * b_values[k].flatten()[E]
            else:
                b_stable[(4*4 + (4*k + j)*E.shape[0]):(4*4 + (4*k + j + 1)*E.shape[0])] = -(1 / biomasses[k]) * x_values[j][J_values[j]:].flatten()[E]
    # Make constraints for partials of R_{k}^{F} \nu_{k}(F_{k}) and R_{k}^{ex} \nu_{k}^{ex}
    # with respect to x_{j}.
    for k in range(4):
        for j in range(4):
            A_kj_row_idx_start = int(4*4 + 4*4*E.shape[0] + np.sum([4*(I_values[w]+Jl) for w in range(k)]) + j*(I_values[k]+Jl))

            block_kj = S[:, lumen_reactions_idx.flatten()]
            block_kj = block_kj[np.concatenate([metabolite_indices[k], lumen_metabolites_idx.flatten()]).flatten(), :]
            A_stable[A_kj_row_idx_start:(A_kj_row_idx_start + I_values[k] + Jl), (4*4 + (4*k + j)*Jl):(4*4 + (4*k + j + 1)*Jl)] = sparse.csr_matrix(block_kj)

            block_kj_internal = S[:, reaction_indices[k][free_var_indices[k]]]
            block_kj_internal = block_kj_internal[np.concatenate([metabolite_indices[k], lumen_metabolites_idx.flatten()]).flatten(), :]
            A_stable[A_kj_row_idx_start:(A_kj_row_idx_start + I_values[k] + Jl),
                     int((4*4 + 4*4*Jl + 4*np.sum([free_var_indices[w].shape[0] for w in range(k)]) + j*free_var_indices[k].shape[0])):int((4*4 + 4*4*Jl + 4*np.sum([free_var_indices[w].shape[0] for w in range(k)]) + (j+1)*free_var_indices[k].shape[0]))] = sparse.csr_matrix(block_kj_internal)

    result = sp_linalg.lsqr(A_stable,b_stable,atol=1e-6,btol=1e-6)[0]
    
    # Row k of h_vals gives values of partials of h_{k}.
    h_vals = np.zeros((4,4))
    for k in range(4):
        h_vals[k,:] = result[k*4:(k+1)*4]
        
    # Now construct the matrix M used to check stability.
    # Row k of M gives values of M_{k,j} for j = 0,1,2,3.
    M = np.zeros((4,4))
    for k in range(4):
        M[k,:] = biomasses[k] * h_vals[k,:]
        
    # Compute eigenvalues of M. If real part of every eigenvalue of 
    # M is less than 0, then we have a stable steady state.
    eigs, vecs = linalg.eig(M)

    if np.max(eigs) > -1e-4:
        stable = False
    else:
        stable = True
    
    max_eig_index = np.argmax(eigs)

    print('eigs: ', eigs)
    print('\n')
    print('vecs: ', vecs)

    return (stable, eigs[max_eig_index], vecs[max_eig_index])
    

In [7]:
total_start_time = time.time()

stable_steady_states = []
unstable_steady_states = []
for i in range(len(steady_states)):
    print('i: ', i)
    target_bm, x1, x2, x3, x4 = steady_states[i]
    try:
        stable, max_eig, _ = stability(x1, target_bm[0], x2, target_bm[1], x3, target_bm[2], x4, target_bm[3], 1e-8)
        if stable:
            stable_steady_states.append((target_bm, x1, x2, x3, x4))
        else:
            unstable_steady_states.append((target_bm, x1, x2, x3, x4))
    except:
        print('\n')
        print('In except')
        print('\n')
        stable, max_eig, _ = stability(x1, target_bm[0], x2, target_bm[1], x3, target_bm[2], x4, target_bm[3], 1e-6)
        if stable:
            stable_steady_states.append((target_bm, x1, x2, x3, x4))
        else:
            unstable_steady_states.append((target_bm, x1, x2, x3, x4))

total_end_time = time.time()


i:  0
Starting stability function


In except


Starting stability function


/home/mitch/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/home/mitch/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:116: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


eigs:  [-1.29698077e-03+0.j  7.46726444e-04+0.j  4.12936176e-06+0.j
  3.10357779e-04+0.j]


vecs:  [[ 0.01125651 -0.86901592  0.2401194   0.69885821]
 [-0.17399316  0.04708676 -0.08062624  0.26565735]
 [-0.98440543  0.45604871  0.63068467 -0.65346413]
 [ 0.02335846 -0.18604772 -0.73353863 -0.11835544]]
i:  1
Starting stability function


In except


Starting stability function
eigs:  [-0.0261376 +0.j -0.00047771+0.j  0.00078069+0.j  0.00054902+0.j]


vecs:  [[-0.3335674  -0.37427564 -0.68806532 -0.05993127]
 [-0.86676376 -0.66658391  0.6752723  -0.78835305]
 [-0.16438951 -0.06821963  0.26250064  0.60895446]
 [-0.33230929  0.641038   -0.04082708 -0.06389188]]
i:  2
Starting stability function
eigs:  [-1.82020368e-02+0.j  2.93404649e-03+0.j -2.67204105e-03+0.j
  2.26804921e-05+0.j]


vecs:  [[ 0.52067047 -0.98174198 -0.85841102 -0.03709364]
 [-0.67732253 -0.04084535  0.45197393 -0.86761837]
 [-0.47632367 -0.12725866  0.24153043  0.48829611]
 [-0.20797167  0.1353498   0.02265275  0.086193

In [9]:
# .p files ending with _040 correspond to problem with dilution rate 0.4, .p files ending with 
# _060 correspond to problem with dilution rate 0.6, and .p files ending with _0018 correspond
# to problem with dilution rate 0.018. No ending corresponds to death rate 0.5.
pickle.dump(stable_steady_states, open("stable_steady_states.p", "wb"))
pickle.dump(unstable_steady_states, open("unstable_steady_states.p", "wb"))


In [1]:
# The following code is useful for looking for qualitatively similar 
# biomass and/or flux profiles among the different steady states.
# This is needed when analyzing a community with many computed 
# stable steady states.
# Plotting the "inertia" of the clustering vs. the number of clusters 
# used can help determine a reasonable number of clusters.
concated_fluxes = []
bms = [stable_steady_states[i][0] for i in range(len(stable_steady_states))]
for i in range(len(stable_steady_states)):
    concated_fluxes.append(
        np.concatenate(
            [
                stable_steady_states[i][1],
                stable_steady_states[i][2],
                stable_steady_states[i][3],
                stable_steady_states[i][4]
            ]
        ).flatten()
    )
from sklearn.cluster import KMeans
inertias = []
for i in range(1, len(concated_fluxes)):
    kmeans = KMeans(n_clusters=i).fit(concated_fluxes)
    inertias.append(kmeans.inertia_)
kmeans = KMeans(n_clusters=5).fit(concated_fluxes)
plt.plot(inertias)


NameError: name 'stable_steady_states' is not defined

In [ ]:
# Once a number of clusters is chosen, can use the cluster centers
# as a representation of the general flux or biomass profile for 
# steady states in that cluster.
m = KMeans(n_clusters=4).fit(bms)
print(m.cluster_centers_)
print('\n')
print(m.labels_)


Now compute whether or not each of the computed 
steady state GNE is stable to the invasion of 
a fifth *E. coli* mutant described in the paper.

In [ ]:
# Load data for five EColi model to look at stability to invasion.
directory = '../ModelFiles/FiveSpecies'

# S contains the stoichiometry matrices R_{k} and R_{k}^{ex} for each species k.
S = sio.loadmat(directory + '/S.mat')['S']
I = sio.loadmat(directory + '/I.mat')['I'][0][0]
J = sio.loadmat(directory + '/J.mat')['J'][0][0]
reaction_lb = sio.loadmat(directory + '/lb.mat')['lb']
reaction_ub = sio.loadmat(directory + '/ub.mat')['ub']

# Indices of reactions and metabolites for each species, needed because 
# cobra groups all metabolites and reactions into a single model.
lumen_reactions_idx = sio.loadmat(directory + '/lumen_reactions_idx.mat')['lumen_reactions_idx'] - 1
lumen_metabolites_idx = sio.loadmat(directory + '/lumen_metabolites_idx.mat')['lumen_metabolites_idx'] - 1
lumen_reaction_names = sio.loadmat(directory + '/lumen_reactions.mat')['lumen_reactions']

Ec1_reactions_idx = sio.loadmat(directory + '/Ec1_reactions_idx.mat')['Ec1_reactions_idx'] - 1
Ec1_reaction_names = sio.loadmat(directory + '/Ec1_reactions.mat')['Ec1_reactions']
Ec1_metabolites_idx = sio.loadmat(directory + '/Ec1_metabolites_idx.mat')['Ec1_metabolites_idx'] - 1
Ec1_biomass_idx = sio.loadmat(directory + '/Ec1_biomass_idx.mat')['Ec1_biomass_idx'][0][0]-1

Ec2_reactions_idx = sio.loadmat(directory + '/Ec2_reactions_idx.mat')['Ec2_reactions_idx'] - 1
Ec2_reaction_names = sio.loadmat(directory + '/Ec2_reactions.mat')['Ec2_reactions']
Ec2_metabolites_idx = sio.loadmat(directory + '/Ec2_metabolites_idx.mat')['Ec2_metabolites_idx'] - 1
Ec2_biomass_idx = sio.loadmat(directory + '/Ec2_biomass_idx.mat')['Ec2_biomass_idx'][0][0]-1

Ec3_reactions_idx = sio.loadmat(directory + '/Ec3_reactions_idx.mat')['Ec3_reactions_idx'] - 1
Ec3_reaction_names = sio.loadmat(directory + '/Ec3_reactions.mat')['Ec3_reactions']
Ec3_metabolites_idx = sio.loadmat(directory + '/Ec3_metabolites_idx.mat')['Ec3_metabolites_idx'] - 1
Ec3_biomass_idx = sio.loadmat(directory + '/Ec3_biomass_idx.mat')['Ec3_biomass_idx'][0][0]-1

Ec4_reactions_idx = sio.loadmat(directory + '/Ec4_reactions_idx.mat')['Ec4_reactions_idx'] - 1
Ec4_reaction_names = sio.loadmat(directory + '/Ec4_reactions.mat')['Ec4_reactions']
Ec4_metabolites_idx = sio.loadmat(directory + '/Ec4_metabolites_idx.mat')['Ec4_metabolites_idx'] - 1
Ec4_biomass_idx = sio.loadmat(directory + '/Ec4_biomass_idx.mat')['Ec4_biomass_idx'][0][0]-1

Ec5_reactions_idx = sio.loadmat(directory + '/Ec5_reactions_idx.mat')['Ec5_reactions_idx'] - 1
Ec5_reaction_names = sio.loadmat(directory + '/Ec5_reactions.mat')['Ec5_reactions']
Ec5_metabolites_idx = sio.loadmat(directory + '/Ec5_metabolites_idx.mat')['Ec5_metabolites_idx'] - 1
Ec5_biomass_idx = sio.loadmat(directory + '/Ec5_biomass_idx.mat')['Ec5_biomass_idx'][0][0]-1

I1 = len(Ec1_metabolites_idx); I2 = len(Ec2_metabolites_idx); I3 = len(Ec3_metabolites_idx); I4 = len(Ec4_metabolites_idx); I5 = len(Ec5_metabolites_idx)
Jl = len(lumen_reactions_idx); J1 = len(Ec1_reactions_idx); J2 = len(Ec2_reactions_idx); J3 = len(Ec3_reactions_idx); J4 = len(Ec4_reactions_idx); J5 = len(Ec5_reactions_idx)


In [ ]:
Ec1_reaction_names = np.array([Ec1_reaction_names[i][0] for i in range(len(Ec1_reaction_names))])
Ec2_reaction_names = np.array([Ec2_reaction_names[i][0] for i in range(len(Ec2_reaction_names))])
Ec3_reaction_names = np.array([Ec3_reaction_names[i][0] for i in range(len(Ec3_reaction_names))])
Ec4_reaction_names = np.array([Ec4_reaction_names[i][0] for i in range(len(Ec4_reaction_names))])
Ec5_reaction_names = np.array([Ec5_reaction_names[i][0] for i in range(len(Ec5_reaction_names))])
lumen_reaction_names = np.array([lumen_reaction_names[i][0] for i in range(len(lumen_reaction_names))])


In [ ]:
# Create vectors that can be dotted with vector of reactions for each species 
# and pull out the biomass reaction.
e1 = sparse.identity(J1 + Jl).tocsr()[:, Ec1_biomass_idx]; e2 = sparse.identity(J2 + Jl).tocsr()[:, Ec2_biomass_idx]
e3 = sparse.identity(J3 + Jl).tocsr()[:, Ec3_biomass_idx]; e4 = sparse.identity(J4 + Jl).tocsr()[:, Ec4_biomass_idx]
e5 = sparse.identity(J5 + Jl).tocsr()[:, Ec5_biomass_idx]


In [ ]:
invaded_steady_states = []
uninvaded_steady_states = []


In [ ]:
def invasion_stability_opt(bm1, bm2, bm3, bm4, B1, B2, B3, B4, E):
    ''' Determines stability of invasion to a new species by solving problem (20) in the main body of the paper.
    '''
    solution = cp.Variable((J5 + Jl, 1))
    objective = e5.T @ solution
    constraints = [S[:, np.concatenate([Ec5_reactions_idx, lumen_reactions_idx]).flatten()] @ solution == 0,
                   solution[0:J5] >= reaction_lb[Ec5_reactions_idx.flatten()],
                   solution[0:J5] <= reaction_ub[Ec5_reactions_idx.flatten()],
                   (sparse.identity(Jl).tocsr() + bm1 * B1 + bm2 * B2 + bm3 * B3 + bm4 * B4)[E, :] @ solution[J5:] >= 0
]
    prob = cp.Problem(cp.Maximize(objective), constraints)
    prob.solve(solver = cp_solver)
    return prob.value
    

In [ ]:
def make_B_mats(x1, x2, x3, x4, bm1, bm2, bm3, bm4, pert_size):
    '''Function for generating the matrices B_k for all species k as given in the proof of Lemma 3.6.'''

    # First, need to calculate B matrices. To do so, need to first calculate the Pi matrices.
    # In order to do \emph{that}, need to calculate the Omega matrices.
    env = gurobipy.Env(empty=True)
    env.setParam("OutputFlag",0)
    env.start()
    
    # Ec1's problem.
    m1 = gurobipy.Model("Ec1", env=env)

    # Create variables.
    lb1 = reaction_lb[Ec1_reactions_idx.flatten()].flatten()
    ub1 = reaction_ub[Ec1_reactions_idx.flatten()].flatten()

    lb1_pert = np.random.random((J1, 1))
    lbl_pert = np.random.random((Jl, 1))
    ub1_pert = np.random.random((J1, 1))

    lb1 = lb1 - pert_size * lb1_pert.flatten()
    ub1 = ub1 + pert_size * ub1_pert.flatten()

    rxns1 = m1.addMVar(shape=J1, name='rxns1', lb=lb1, ub=ub1);

    lb1_ex = (reaction_lb[lumen_reactions_idx.flatten()].flatten() - bm2 * x2[J2:].flatten() - bm3 * x3[J3:].flatten() - bm4 * x4[J4:].flatten()) / bm1
    ub1_ex = (reaction_ub[lumen_reactions_idx.flatten()].flatten() - bm2 * x2[J2:].flatten() - bm3 * x3[J3:].flatten() - bm4 * x4[J4:].flatten()) / bm1

    lb1_ex = lb1_ex - pert_size * lbl_pert.flatten()

    rxns1_ex = m1.addMVar(shape=Jl, name='rxns1_ex', lb=lb1_ex, ub=ub1_ex);

    # Set objective.
    m1.setObjective(e1.toarray()[0:J1].T @ rxns1, gurobipy.GRB.MAXIMIZE);

    # Make constraints.
    S1 = S[np.concatenate([Ec1_metabolites_idx, lumen_metabolites_idx]).flatten(), :]
    S1_ex = S1[:, lumen_reactions_idx.flatten()]
    S1 = S1[:, Ec1_reactions_idx.flatten()]
    m1.addConstr(S1 @ rxns1 + S1_ex @ rxns1_ex == np.zeros((S1.shape[0],)), name='internal_fba1');

    m1.params.Method = 0;
    m1.update();
    m1.optimize();

    # Ec2's problem.
    m2 = gurobipy.Model("Ec2", env=env)

    # Create variables.
    lb2 = reaction_lb[Ec2_reactions_idx.flatten()].flatten()
    ub2 = reaction_ub[Ec2_reactions_idx.flatten()].flatten()

    lb2_pert = np.random.random((J2, 1))
    lbl_pert = np.random.random((Jl, 1))
    ub2_pert = np.random.random((J2, 1))

    lb2 = lb2 - pert_size * lb2_pert.flatten()
    ub2 = ub2 + pert_size * ub2_pert.flatten()

    rxns2 = m2.addMVar(shape=J2, name='rxns2', lb=lb2, ub=ub2);

    lb2_ex = (reaction_lb[lumen_reactions_idx.flatten()].flatten() - bm1 * x1[J1:].flatten() - bm3 * x3[J3:].flatten() - bm4 * x4[J4:].flatten()) / bm2
    ub2_ex = (reaction_ub[lumen_reactions_idx.flatten()].flatten() - bm1 * x1[J1:].flatten() - bm3 * x3[J3:].flatten() - bm4 * x4[J4:].flatten()) / bm2

    lb2_ex = lb2_ex - pert_size * lbl_pert.flatten()

    rxns2_ex = m2.addMVar(shape=Jl, name='rxns2_ex', lb=lb2_ex, ub=ub2_ex);

    # Set objective.
    m2.setObjective(e2.toarray()[0:J2].T @ rxns2, gurobipy.GRB.MAXIMIZE);

    # Make constraints.
    S2 = S[np.concatenate([Ec2_metabolites_idx, lumen_metabolites_idx]).flatten(), :]
    S2_ex = S2[:, lumen_reactions_idx.flatten()]
    S2 = S2[:, Ec2_reactions_idx.flatten()]
    m2.addConstr(S2 @ rxns2 + S2_ex @ rxns2_ex == np.zeros((S2.shape[0],)), name='internal_fba2');

    m2.params.Method = 0;
    m2.update();
    m2.optimize();

    # Ec3's problem.
    m3 = gurobipy.Model("Ec3", env=env)

    # Create variables.
    lb3 = reaction_lb[Ec3_reactions_idx.flatten()].flatten()
    ub3 = reaction_ub[Ec3_reactions_idx.flatten()].flatten()

    lb3_pert = np.random.random((J3,1))
    lbl_pert = np.random.random((Jl,1))
    ub3_pert = np.random.random((J3,1))

    lb3 = lb3 - pert_size * lb3_pert.flatten()
    ub3 = ub3 + pert_size * ub3_pert.flatten()

    rxns3 = m3.addMVar(shape=J3, name='rxns3', lb=lb3, ub=ub3);

    lb3_ex = (reaction_lb[lumen_reactions_idx.flatten()].flatten() - bm2 * x2[J2:].flatten() - bm1 * x1[J1:].flatten() - bm4 * x4[J4:].flatten()) / bm3
    ub3_ex = (reaction_ub[lumen_reactions_idx.flatten()].flatten() - bm2 * x2[J2:].flatten() - bm1 * x1[J1:].flatten() - bm4 * x4[J4:].flatten()) / bm3

    lb3_ex = lb3_ex - pert_size * lbl_pert.flatten()

    rxns3_ex = m3.addMVar(shape=Jl, name='rxns3_ex', lb=lb3_ex, ub=ub3_ex);

    # Set objective.
    m3.setObjective(e3.toarray()[0:J3].T @ rxns3, gurobipy.GRB.MAXIMIZE);

    # Make constraints.
    S3 = S[np.concatenate([Ec3_metabolites_idx, lumen_metabolites_idx]).flatten(), :]
    S3_ex = S3[:, lumen_reactions_idx.flatten()]
    S3 = S3[:, Ec3_reactions_idx.flatten()]
    m3.addConstr(S3 @ rxns3 + S3_ex @ rxns3_ex == np.zeros((S3.shape[0],)), name='internal_fba3');

    m3.params.Method = 0;
    m3.update();
    m3.optimize();

    # Ec4's problem.
    m4 = gurobipy.Model("Ec4", env=env)

    # Create variables.
    lb4 = reaction_lb[Ec4_reactions_idx.flatten()].flatten()
    ub4 = reaction_ub[Ec4_reactions_idx.flatten()].flatten()

    lb4_pert = np.random.random((J4,1))
    lbl_pert = np.random.random((Jl,1))
    ub4_pert = np.random.random((J4,1))

    lb4 = lb4 - pert_size * lb4_pert.flatten()
    ub4 = ub4 + pert_size * ub4_pert.flatten()

    rxns4 = m4.addMVar(shape=J4, name='rxns4', lb=lb4, ub=ub4);

    lb4_ex = (reaction_lb[lumen_reactions_idx.flatten()].flatten() - bm2 * x2[J2:].flatten() - bm3 * x3[J3:].flatten() - bm1 * x1[J1:].flatten()) / bm4
    ub4_ex = (reaction_ub[lumen_reactions_idx.flatten()].flatten() - bm2 * x2[J2:].flatten() - bm3 * x3[J3:].flatten() - bm1 * x1[J1:].flatten()) / bm4

    lb4_ex = lb4_ex - pert_size * lbl_pert.flatten()

    rxns4_ex = m4.addMVar(shape=Jl, name='rxns4_ex', lb=lb4_ex, ub=ub4_ex);

    # Set objective.
    m4.setObjective(e4.toarray()[0:J4].T @ rxns4, gurobipy.GRB.MAXIMIZE);

    # Make constraints.
    S4 = S[np.concatenate([Ec4_metabolites_idx, lumen_metabolites_idx]).flatten(), :]
    S4_ex = S4[:, lumen_reactions_idx.flatten()]
    S4 = S4[:, Ec4_reactions_idx.flatten()]
    m4.addConstr(S4 @ rxns4 + S4_ex @ rxns4_ex == np.zeros((S4.shape[0],)), name='internal_fba4');

    m4.params.Method = 0;
    m4.update();
    m4.optimize();
    
    # Check degeneracy.
    primal_basis1 = rxns1.VBasis == 0
    primal_basis1_ex = rxns1_ex.VBasis == 0
    nonbasic_constraints1 = np.array(m1.CBasis) != 0

    active_basic1_lb = np.where(rxns1.X[primal_basis1] - lb1[primal_basis1] == 0)[0]
    active_basic1_ub = np.where(ub1[primal_basis1] - rxns1.X[primal_basis1] == 0)[0]
    active_basic1_ex_lb = np.where(rxns1_ex.X[primal_basis1_ex] - lb1_ex[primal_basis1_ex] == 0)[0]
    active_basic1_ex_ub = np.where(ub1_ex[primal_basis1_ex] - rxns1_ex.X[primal_basis1_ex] == 0)[0]

    if len(active_basic1_lb) == 0 and len(active_basic1_ub) == 0 and len(active_basic1_ex_lb) == 0 and len(active_basic1_ex_ub) == 0:
        degenerate = False
    else:
        degenerate = True

    if degenerate:
        raise Exception('Ec1 problem is degenerate')

    primal_basis2 = rxns2.VBasis == 0
    primal_basis2_ex = rxns2_ex.VBasis == 0
    nonbasic_constraints2 = np.array(m2.CBasis) != 0

    active_basic2_lb = np.where(rxns2.X[primal_basis2] - lb2[primal_basis2] == 0)[0]
    active_basic2_ub = np.where(ub2[primal_basis2] - rxns2.X[primal_basis2] == 0)[0]
    active_basic2_ex_lb = np.where(rxns2_ex.X[primal_basis2_ex] - lb2_ex[primal_basis2_ex] == 0)[0]
    active_basic2_ex_ub = np.where(ub2_ex[primal_basis2_ex] - rxns2_ex.X[primal_basis2_ex] == 0)[0]

    if len(active_basic2_lb) == 0 and len(active_basic2_ub) == 0 and len(active_basic2_ex_lb) == 0 and len(active_basic2_ex_ub) == 0:
        degenerate = False
    else:
        degenerate = True

    if degenerate:
        raise Exception('Ec2 problem is degenerate')

    primal_basis3 = rxns3.VBasis == 0
    primal_basis3_ex = rxns3_ex.VBasis == 0
    nonbasic_constraints3 = np.array(m3.CBasis) != 0

    active_basic3_lb = np.where(rxns3.X[primal_basis3] - lb3[primal_basis3] == 0)[0]
    active_basic3_ub = np.where(ub3[primal_basis3] - rxns3.X[primal_basis3] == 0)[0]
    active_basic3_ex_lb = np.where(rxns3_ex.X[primal_basis3_ex] - lb3_ex[primal_basis3_ex] == 0)[0]
    active_basic3_ex_ub = np.where(ub3_ex[primal_basis3_ex] - rxns3_ex.X[primal_basis3_ex] == 0)[0]

    if len(active_basic3_lb) == 0 and len(active_basic3_ub) == 0 and len(active_basic3_ex_lb) == 0 and len(active_basic3_ex_ub) == 0:
        degenerate = False
    else:
        degenerate = True

    if degenerate:
        raise Exception('Ec3 problem is degenerate')

    primal_basis4 = rxns4.VBasis == 0
    primal_basis4_ex = rxns4_ex.VBasis == 0
    nonbasic_constraints4 = np.array(m4.CBasis) != 0

    active_basic4_lb = np.where(rxns4.X[primal_basis4] - lb4[primal_basis4] == 0)[0]
    active_basic4_ub = np.where(ub4[primal_basis4] - rxns4.X[primal_basis4] == 0)[0]
    active_basic4_ex_lb = np.where(rxns4_ex.X[primal_basis4_ex] - lb4_ex[primal_basis4_ex] == 0)[0]
    active_basic4_ex_ub = np.where(ub4_ex[primal_basis4_ex] - rxns4_ex.X[primal_basis4_ex] == 0)[0]

    if len(active_basic4_lb) == 0 and len(active_basic4_ub) == 0 and len(active_basic4_ex_lb) == 0 and len(active_basic4_ex_ub) == 0:
        degenerate = False
    else:
        degenerate = True

    if degenerate:
        raise Exception('Ec4 problem is degenerate')
        
    U1 = np.where(ub1 - rxns1.X == 0)[0]
    L1 = np.where(rxns1.X - lb1 == 0)[0]
    F1 = np.where(np.logical_and(~np.isin(np.arange(J1), U1), ~np.isin(np.arange(J1), L1)))[0]
    L1_ex = np.where(rxns1_ex.X - lb1_ex == 0)[0]
    F1_ex = np.where(~np.isin(np.arange(Jl), L1_ex))[0]

    U2 = np.where(ub2 - rxns2.X == 0)[0]
    L2 = np.where(rxns2.X - lb2 == 0)[0]
    F2 = np.where(np.logical_and(~np.isin(np.arange(J2), U2), ~np.isin(np.arange(J2), L2)))[0]
    L2_ex = np.where(rxns2_ex.X - lb2_ex == 0)[0]
    F2_ex = np.where(~np.isin(np.arange(Jl), L2_ex))[0]

    U3 = np.where(ub3 - rxns3.X == 0)[0]
    L3 = np.where(rxns3.X - lb3 == 0)[0]
    F3 = np.where(np.logical_and(~np.isin(np.arange(J3), U3), ~np.isin(np.arange(J3), L3)))[0]
    L3_ex = np.where(rxns3_ex.X - lb3_ex == 0)[0]
    F3_ex = np.where(~np.isin(np.arange(Jl), L3_ex))[0]

    U4 = np.where(ub4 - rxns4.X == 0)[0]
    L4 = np.where(rxns4.X - lb4 == 0)[0]
    F4 = np.where(np.logical_and(~np.isin(np.arange(J4), U4), ~np.isin(np.arange(J4), L4)))[0]
    L4_ex = np.where(rxns4_ex.X - lb4_ex == 0)[0]
    F4_ex = np.where(~np.isin(np.arange(Jl), L4_ex))[0]
    
    Omega1 = sparse.csr_matrix((sum(nonbasic_constraints1) + len(L1) + len(U1) + len(L1_ex), J1 + Jl))
    Omega2 = sparse.csr_matrix((sum(nonbasic_constraints2) + len(L2) + len(U2) + len(L2_ex), J2 + Jl))
    Omega3 = sparse.csr_matrix((sum(nonbasic_constraints3) + len(L3) + len(U3) + len(L3_ex), J3 + Jl))
    Omega4 = sparse.csr_matrix((sum(nonbasic_constraints4) + len(L4) + len(U4) + len(L4_ex), J4 + Jl))

    Omega1[0:sum(nonbasic_constraints1), 0:len(L1)] = S1[nonbasic_constraints1, :][:, L1]
    Omega2[0:sum(nonbasic_constraints2), 0:len(L2)] = S2[nonbasic_constraints2, :][:, L2]
    Omega3[0:sum(nonbasic_constraints3), 0:len(L3)] = S3[nonbasic_constraints3, :][:, L3]
    Omega4[0:sum(nonbasic_constraints4), 0:len(L4)] = S4[nonbasic_constraints4, :][:, L4]

    Omega1[0:sum(nonbasic_constraints1), len(L1):len(L1)+len(U1)] = S1[nonbasic_constraints1, :][:, U1]
    Omega2[0:sum(nonbasic_constraints2), len(L2):len(L2)+len(U2)] = S2[nonbasic_constraints2, :][:, U2]
    Omega3[0:sum(nonbasic_constraints3), len(L3):len(L3)+len(U3)] = S3[nonbasic_constraints3, :][:, U3]
    Omega4[0:sum(nonbasic_constraints4), len(L4):len(L4)+len(U4)] = S4[nonbasic_constraints4, :][:, U4]

    Omega1[0:sum(nonbasic_constraints1), len(L1)+len(U1):len(L1)+len(U1)+len(F1)] = S1[nonbasic_constraints1, :][:, F1]
    Omega2[0:sum(nonbasic_constraints2), len(L2)+len(U2):len(L2)+len(U2)+len(F2)] = S2[nonbasic_constraints2, :][:, F2]
    Omega3[0:sum(nonbasic_constraints3), len(L3)+len(U3):len(L3)+len(U3)+len(F3)] = S3[nonbasic_constraints3, :][:, F3]
    Omega4[0:sum(nonbasic_constraints4), len(L4)+len(U4):len(L4)+len(U4)+len(F4)] = S4[nonbasic_constraints4, :][:, F4]

    Omega1[0:sum(nonbasic_constraints1), len(L1)+len(U1)+len(F1):len(L1)+len(U1)+len(F1)+len(L1_ex)] = S1_ex[nonbasic_constraints1, :][:, L1_ex]
    Omega2[0:sum(nonbasic_constraints2), len(L2)+len(U2)+len(F2):len(L2)+len(U2)+len(F2)+len(L2_ex)] = S2_ex[nonbasic_constraints2, :][:, L2_ex]
    Omega3[0:sum(nonbasic_constraints3), len(L3)+len(U3)+len(F3):len(L3)+len(U3)+len(F3)+len(L3_ex)] = S3_ex[nonbasic_constraints3, :][:, L3_ex]
    Omega4[0:sum(nonbasic_constraints4), len(L4)+len(U4)+len(F4):len(L4)+len(U4)+len(F4)+len(L4_ex)] = S4_ex[nonbasic_constraints4, :][:, L4_ex]

    Omega1[0:sum(nonbasic_constraints1), len(L1)+len(U1)+len(F1)+len(L1_ex):] = S1_ex[nonbasic_constraints1, :][:, F1_ex]
    Omega2[0:sum(nonbasic_constraints2), len(L2)+len(U2)+len(F2)+len(L2_ex):] = S2_ex[nonbasic_constraints2, :][:, F2_ex]
    Omega3[0:sum(nonbasic_constraints3), len(L3)+len(U3)+len(F3)+len(L3_ex):] = S3_ex[nonbasic_constraints3, :][:, F3_ex]
    Omega4[0:sum(nonbasic_constraints4), len(L4)+len(U4)+len(F4)+len(L4_ex):] = S4_ex[nonbasic_constraints4, :][:, F4_ex]

    Omega1[sum(nonbasic_constraints1):sum(nonbasic_constraints1)+len(L1), 0:len(L1)] = np.identity(len(L1))
    Omega2[sum(nonbasic_constraints2):sum(nonbasic_constraints2)+len(L2), 0:len(L2)] = np.identity(len(L2))
    Omega3[sum(nonbasic_constraints3):sum(nonbasic_constraints3)+len(L3), 0:len(L3)] = np.identity(len(L3))
    Omega4[sum(nonbasic_constraints4):sum(nonbasic_constraints4)+len(L4), 0:len(L4)] = np.identity(len(L4))

    Omega1[sum(nonbasic_constraints1)+len(L1):sum(nonbasic_constraints1)+len(L1)+len(U1), len(L1):len(L1)+len(U1)] = np.identity(len(U1))
    Omega2[sum(nonbasic_constraints2)+len(L2):sum(nonbasic_constraints2)+len(L2)+len(U2), len(L2):len(L2)+len(U2)] = np.identity(len(U2))
    Omega3[sum(nonbasic_constraints3)+len(L3):sum(nonbasic_constraints3)+len(L3)+len(U3), len(L3):len(L3)+len(U3)] = np.identity(len(U3))
    Omega4[sum(nonbasic_constraints4)+len(L4):sum(nonbasic_constraints4)+len(L4)+len(U4), len(L4):len(L4)+len(U4)] = np.identity(len(U4))

    Omega1[sum(nonbasic_constraints1)+len(L1)+len(U1):, len(L1)+len(U1)+len(F1):len(L1)+len(U1)+len(F1)+len(L1_ex)] = np.identity(len(L1_ex))
    Omega2[sum(nonbasic_constraints2)+len(L2)+len(U2):, len(L2)+len(U2)+len(F2):len(L2)+len(U2)+len(F2)+len(L2_ex)] = np.identity(len(L2_ex))
    Omega3[sum(nonbasic_constraints3)+len(L3)+len(U3):, len(L3)+len(U3)+len(F3):len(L3)+len(U3)+len(F3)+len(L3_ex)] = np.identity(len(L3_ex))
    Omega4[sum(nonbasic_constraints4)+len(L4)+len(U4):, len(L4)+len(U4)+len(F4):len(L4)+len(U4)+len(F4)+len(L4_ex)] = np.identity(len(L4_ex))
    
    # Now construct Pi matrices.
    Pi1 = sp_linalg.inv(Omega1)
    Pi2 = sp_linalg.inv(Omega2)
    Pi3 = sp_linalg.inv(Omega3)
    Pi4 = sp_linalg.inv(Omega4)

    Pi1 = Pi1[np.concatenate([L1_ex, F1_ex]).flatten(), :]
    Pi2 = Pi2[np.concatenate([L2_ex, F2_ex]).flatten(), :]
    Pi3 = Pi3[np.concatenate([L3_ex, F3_ex]).flatten(), :]
    Pi4 = Pi4[np.concatenate([L4_ex, F4_ex]).flatten(), :]

    Pi1 = Pi1[:, L1_ex.flatten()]
    Pi2 = Pi2[:, L2_ex.flatten()]
    Pi3 = Pi3[:, L3_ex.flatten()]
    Pi4 = Pi4[:, L4_ex.flatten()]
    
    A = sparse.identity(4*Jl).tocsr()
    B_rhs = sparse.csr_matrix((4*Jl, Jl))
    
    # Make first row of A.
    A[0:Jl, Jl:2*Jl] = (bm2 / bm1) * Pi1 * sparse.identity(Jl).tocsr()[L1_ex, :]
    A[0:Jl, 2*Jl:3*Jl] = (bm3 / bm1) * Pi1 * sparse.identity(Jl).tocsr()[L1_ex, :]
    A[0:Jl, 3*Jl:] = (bm4 / bm1) * Pi1 * sparse.identity(Jl).tocsr()[L1_ex, :]

    # Make second row of A.
    A[Jl:2*Jl, 0:Jl] = (bm1 / bm2) * Pi2 * sparse.identity(Jl).tocsr()[L2_ex, :]
    A[Jl:2*Jl, 2*Jl:3*Jl] = (bm3 / bm2) * Pi2 * sparse.identity(Jl).tocsr()[L2_ex, :]
    A[Jl:2*Jl, 3*Jl:] = (bm4 / bm2) * Pi2 * sparse.identity(Jl).tocsr()[L2_ex, :]

    # Make third row of A.
    A[2*Jl:3*Jl, 0:Jl] = (bm1 / bm3) * Pi3 * sparse.identity(Jl).tocsr()[L3_ex, :]
    A[2*Jl:3*Jl, Jl:2*Jl] = (bm2 / bm3) * Pi3 * sparse.identity(Jl).tocsr()[L3_ex, :]
    A[2*Jl:3*Jl, 3*Jl:] = (bm4 / bm3) * Pi3 * sparse.identity(Jl).tocsr()[L3_ex, :]

    # Make fourth row of A.
    A[3*Jl:, 0:Jl] = (bm1 / bm4) * Pi4 * sparse.identity(Jl).tocsr()[L4_ex, :]
    A[3*Jl:, Jl:2*Jl] = (bm2 / bm4) * Pi4 * sparse.identity(Jl).tocsr()[L4_ex, :]
    A[3*Jl:, 2*Jl:3*Jl] = (bm3 / bm4) * Pi4 * sparse.identity(Jl).tocsr()[L4_ex, :]

    # Add to right-hand side B.
    B_rhs[0:Jl,:] = -(1/bm1) * Pi1 * sparse.identity(Jl).tocsr()[L1_ex, :]
    B_rhs[Jl:2*Jl,:] = -(1/bm2) * Pi2 * sparse.identity(Jl).tocsr()[L2_ex, :]
    B_rhs[2*Jl:3*Jl,:] = -(1/bm3) * Pi3 * sparse.identity(Jl).tocsr()[L3_ex, :]
    B_rhs[3*Jl:, :] = -(1/bm4) * Pi4 * sparse.identity(Jl).tocsr()[L4_ex, :]

    # Now solve for the B matrices.
    B = sp_linalg.spsolve(A, B_rhs)
    B1 = B[0:Jl, :]
    B2 = B[Jl:2*Jl, :]
    B3 = B[2*Jl:3*Jl, :]
    B4 = B[3*Jl:, :]
    
    return (B1, B2, B3, B4)

  

In [ ]:
total_start_time = time.time()

for i in range(len(steady_states)):
    print('i: ', i)
    biomasses, x1, x2, x3, x4 = steady_states[i]
    bm1, bm2, bm3, bm4 = biomasses
    try:
        B1, B2, B3, B4 = make_B_mats(x1, x2, x3, x4, bm1, bm2, bm3, bm4, 1e-8)
        E = np.where(bm1 * x1[J1:] + bm2 * x2[J2:] + bm3 * x3[J3:] + bm4 * x4[J4:] - reaction_lb[lumen_reactions_idx.flatten()] < 1e-6)[0]
        inv_val = invasion_stability_opt(bm1, bm2, bm3, bm4, B1, B2, B3, B4, E)
        if inv_val < death_rate:
            uninvaded_steady_states.append(steady_states[i])
        else:
            invaded_steady_states.append(steady_states[i])
    except:
        print('\n')
        print('in except')
        print('\n')
        B1, B2, B3, B4 = make_B_mats(x1, x2, x3, x4, bm1, bm2, bm3, bm4, 1e-6)
        E = np.where(bm1 * x1[J1:] + bm2 * x2[J2:] + bm3 * x3[J3:] + bm4 * x4[J4:] - reaction_lb[lumen_reactions_idx.flatten()] < 1e-6)[0]
        inv_val = invasion_stability_opt(bm1, bm2, bm3, bm4, B1, B2, B3, B4, E)
        if inv_val < death_rate:
            uninvaded_steady_states.append(steady_states[i])
        else:
            invaded_steady_states.append(steady_states[i])

total_end_time = time.time()


In [ ]:
# .p files ending with _040 correspond to problem with dilution rate 0.4, .p files ending with 
# _060 correspond to problem with dilution rate 0.6, and .p files ending with _0018 correspond
# to problem with dilution rate 0.0018. No ending corresponds to death rate 0.5.
pickle.dump(invaded_steady_states, open("invaded_steady_states.p", "wb"))
pickle.dump(uninvaded_steady_states, open("uninvaded_steady_states.p", "wb"))
